In [2]:
import graphlab as gl
import pandas as pd
import numpy as np

In [116]:
#File location
items = "../BX-CSV-Dump/bookRatingsWithGoogleFeatures.csv"
bookRatingsFile = "../BX-CSV-Dump/BX-Book-Ratings.csv"
bookRatingsFileWithZeros = "../BX-CSV-Dump/ratingsCleanedWithNAs.csv"


In [43]:
dfRating = pd.read_csv(bookRatingsFile)
items = pd.read_csv(items)



In [44]:
#55655 total records with google book info
items[items['_merge'] != 'left_only'].count()
uniqueISBNs = items[items['_merge'] != 'left_only']['ISBN'].unique()
items = items[items['_merge'] != 'left_only']
items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55655 entries, 1467 to 455357
Data columns (total 17 columns):
UserID            55655 non-null int64
ISBN              55655 non-null object
BookRating        55655 non-null int64
isbn              55655 non-null object
title             55655 non-null object
authors           55026 non-null object
publishedDate     55240 non-null object
publisher         53853 non-null object
categories        55627 non-null object
pageCount         54839 non-null float64
averageRating     55064 non-null float64
ratingsCount      55064 non-null float64
maturityRating    55655 non-null object
imageThumbnail    54949 non-null object
infoLink          55655 non-null object
textSnippet       54379 non-null object
_merge            55655 non-null object
dtypes: float64(3), int64(2), object(12)
memory usage: 7.6+ MB


In [45]:
len(uniqueISBNs)

847

In [46]:
# 21050 = total count of ratings for the subset of ISBNs gathered from Google books lookup
print dfRating.info()
dfRating[dfRating['ISBN'].isin(uniqueISBNs)].count()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176019 entries, 0 to 176018
Data columns (total 3 columns):
UserID        176019 non-null int64
ISBN          176019 non-null object
BookRating    176019 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.0+ MB
None


UserID        21050
ISBN          21050
BookRating    21050
dtype: int64

In [47]:
dfRating[dfRating['ISBN'].isin(uniqueISBNs)].count()

UserID        21050
ISBN          21050
BookRating    21050
dtype: int64

In [48]:
#Train recommendation system with ISBNs for which we were able to get Google lookups successfully
print dfRating.info()
dfRating = dfRating[dfRating['ISBN'].isin(uniqueISBNs)]
print "\n"
dfRating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176019 entries, 0 to 176018
Data columns (total 3 columns):
UserID        176019 non-null int64
ISBN          176019 non-null object
BookRating    176019 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.0+ MB
None


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21050 entries, 14 to 175990
Data columns (total 3 columns):
UserID        21050 non-null int64
ISBN          21050 non-null object
BookRating    21050 non-null int64
dtypes: int64(2), object(1)
memory usage: 657.8+ KB


In [118]:
dfRatingsWithZeroes = pd.read_csv(bookRatingsFileWithZeros)
dfRatingsWithZeroes = dfRatingsWithZeroes[dfRatingsWithZeroes['ISBN'].isin(uniqueISBNs)]
print "\n"
dfRatingsWithZeroes.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 51310 entries, 11 to 493801
Data columns (total 3 columns):
UserID        51310 non-null int64
ISBN          51310 non-null object
BookRating    51310 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.6+ MB


In [120]:
dfRatingsWithZeroes = dfRatingsWithZeroes[dfRatingsWithZeroes['BookRating'] == 0]

In [49]:
#convert subset ratings table to GraphLab SFrame
dfRating = gl.SFrame(dfRating)

In [121]:
dfRatingsWithZeroes = gl.SFrame(dfRatingsWithZeroes)

In [50]:
dfRating.column_types

<bound method SFrame.column_types of Columns:
	UserID	int
	ISBN	str
	BookRating	int

Rows: 21050

Data:
+--------+------------+------------+
| UserID |    ISBN    | BookRating |
+--------+------------+------------+
| 276754 | 0684867621 |     8      |
| 276755 | 0451166892 |     5      |
| 276788 | 043935806X |     7      |
| 276804 | 0440498058 |     8      |
| 276822 | 0060096195 |     10     |
| 276822 | 0786817070 |     10     |
| 276830 | 0440225825 |     9      |
| 276832 | 0345446860 |     5      |
| 276896 | 0440241537 |     10     |
| 276925 | 002542730X |     10     |
+--------+------------+------------+
[21050 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.>

In [114]:
dfRating[dfRating['BookRating'] != 0]

UserID,ISBN,BookRating
276754,0684867621,8
276755,0451166892,5
276788,043935806X,7
276804,0440498058,8
276822,0060096195,10
276822,0786817070,10
276830,0440225825,9
276832,0345446860,5
276896,0440241537,10
276925,002542730X,10


In [133]:
#Save items subset data to local file, for reloading as an SFrame
#columnFields = ['UserID','ISBN','BookRating','title','authors','publishedDate',\
#                'publisher','categories','pageCount','imageThumbnail','textSnippet']
#items = items[columnFields]
#items.info()
#items.to_csv("../BX-CSV-Dump/itemsSubsetForGraphLab.csv")

items = gl.SFrame.read_csv("../BX-CSV-Dump/itemsSubsetForGraphLab.csv",\
                           column_type_hints={"ISBN":str, "pageCount":int})



Finished parsing file /Users/jhonasttanregalado/Documents/DataScience/bootcamp7/githubcrush/bootcamp007_project/Project5-Capstone/shopaholics/BX-CSV-Dump/itemsSubsetForGraphLab.csv

Parsing completed. Parsed 100 lines in 0.44963 secs.

These column type hints were not used: X1

Finished parsing file /Users/jhonasttanregalado/Documents/DataScience/bootcamp7/githubcrush/bootcamp007_project/Project5-Capstone/shopaholics/BX-CSV-Dump/itemsSubsetForGraphLab.csv

Parsing completed. Parsed 55655 lines in 0.406617 secs.

In [134]:
#prep dataframe for unique items only - original Rows: 55655

items.remove_columns(['X1','UserID', 'BookRating'])
items = items.unique()
items.column_types

<bound method SFrame.column_types of Columns:
	ISBN	str
	authors	str
	categories	str
	imageThumbnail	str
	pageCount	int
	publishedDate	str
	publisher	str
	textSnippet	str
	title	str

Rows: 883

Data:
+------------+-------------------------------+---------------------------+
|    ISBN    |            authors            |         categories        |
+------------+-------------------------------+---------------------------+
| 051512317X |          Nora Roberts         |          Fiction          |
| 0440204194 |          Maeve Binchy         |          Fiction          |
| 1573229725 |          Sarah Waters         |          Fiction          |
| 0689716958 |       Virginia Hamilton       |      Juvenile Fiction     |
| 0449003795 |          Sue Grafton          |          Fiction          |
| 0767902521 |          Bill Bryson          |           Travel          |
| 044691200X | MAD Magazine, Sergio Aragonés |           Humor           |
| 068484267X |         Frank McCourt         | Bio

In [123]:
print dfRating['UserID']
print "\n"
dfRatingsWithZeroes['UserID']

[276754, 276755, 276788, 276804, 276822, 276822, 276830, 276832, 276896, 276925, 276925, 276957, 276964, 276964, 276986, 276986, 277042, 277042, 277042, 277051, 277072, 277096, 277107, 277124, 277124, 277124, 277143, 277157, 277171, 277179, 277195, 277195, 277197, 277212, 277229, 277235, 277235, 277237, 277246, 277261, 277315, 277358, 277364, 277378, 277378, 277378, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277439, 277439, 277462, 277466, 277466, 277469, 277517, 277523, 277523, 277523, 277523, 277523, 277523, 277523, 277527, 277527, 277527, 277529, 277533, 277533, 277535, 277554, 277554, 277575, 277575, 277589, 277598, 277601, 277623, 277623, 277623, 277629, 277629, 277629, 277629, 277639, 277639, 277639, 277639, 277639, 277647, 277651, ... ]




dtype: int
Rows: 30260
[276746, 276746, 276762, 276847, 276925, 276925, 276929, 276929, 276939, 276939, 276954, 276964, 276989, 277042, 277042, 277042, 277042, 277042, 277051, 277114, 277143, 277143, 277157, 277157, 277157, 277157, 277168, 277168, 277168, 277168, 277170, 277170, 277171, 277187, 277187, 277195, 277195, 277195, 277195, 277195, 277195, 277195, 277195, 277195, 277195, 277195, 277195, 277195, 277195, 277195, 277197, 277231, 277244, 277245, 277272, 277288, 277378, 277378, 277378, 277378, 277378, 277378, 277411, 277413, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, 277427, ... ]

In [10]:
items['ISBN']

dtype: str
Rows: 498156
['0330299891', '0375404120', '#6612432', "'9607092856'", "'9607092910'", '(THEWINDMILLP', ')440206529', '*0515128325', '+0451197399', '0 00 612183 7', '0 140362525', '0 586 05153 8', '0 671 74251 5', '0 7475 3274 5', '0 7525 1962 x', '0 85550 000 x', '0 907 062 008', '0 907 062 008', '0 907 062 008', '0*708880258', '0*708880258', '0.330241664', '0.380.50377.8', '0/330/25864/8', '00.55339', '00000000', '00000000', '000000000', '000000000', '000000000', '0000000000', '000000000000', '0000000000000', '0000000000000', '0000000008573', '0000000022057', '0000000029841', '0000000029841', '0000000029841', '0000000030670', '0000000033220', '0000000054690', '0000001042283', '0000001042283', '0000001077469', '0000001481', '000001246971', '000001246971', '000003827X', '000003827X', '0000094072', '0000121214523', '0000225820X', '00006512054', '00006750184', '00007167040', '0000722105029', '0000913154', '0001010565', '0001016237', '0001033085', '0001046713', '0001046934', '00

In [67]:
#variable to be referenced in training model section
actions = dfRating

In [68]:
actions.column_names

<bound method SFrame.column_names of Columns:
	UserID	int
	ISBN	str
	BookRating	int

Rows: 21050

Data:
+--------+------------+------------+
| UserID |    ISBN    | BookRating |
+--------+------------+------------+
| 276754 | 0684867621 |     8      |
| 276755 | 0451166892 |     5      |
| 276788 | 043935806X |     7      |
| 276804 | 0440498058 |     8      |
| 276822 | 0060096195 |     10     |
| 276822 | 0786817070 |     10     |
| 276830 | 0440225825 |     9      |
| 276832 | 0345446860 |     5      |
| 276896 | 0440241537 |     10     |
| 276925 | 002542730X |     10     |
+--------+------------+------------+
[21050 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.>

### Examples

Basic usage

Given basic user-item observation data, an ItemSimilarityRecommender is created:

>>> sf = graphlab.SFrame({'user_id': ['0', '0', '0', '1', '1', '2', '2', '2'],
...                       'item_id': ['a', 'b', 'c', 'a', 'b', 'b', 'c', 'd']})
>>> m = graphlab.recommender.create(sf)
>>> recs = m.recommend()
Creating a model for ratings data

This trains a FactorizationRecommender that can predict target ratings:

>>> sf2 = graphlab.SFrame({'user_id': ['0', '0', '0', '1', '1', '2', '2', '2'],
...                        'item_id': ['a', 'b', 'c', 'a', 'b', 'b', 'c', 'd'],
...                        'rating': [1, 3, 2, 5, 4, 1, 4, 3]})
>>> m2 = graphlab.recommender.create(sf2, target="rating", ranking = False)


In [69]:
#build a model - defaults to jaccardian
training_data, validation_data = gl.recommender.util.random_split_by_user(actions, 'UserID', 'ISBN')
model = gl.recommender.create(training_data, 'UserID', 'ISBN')

Recsys training: model = item_similarity

Warning: Column 'BookRating' ignored.

To use this column as the target, set target = "BookRating" and use a method that allows the use of a target.

Preparing data set.

Data has 20557 observations with 8709 users and 826 items.

...

| 25.614ms                            | 100              | 826             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.04602s

In [70]:
results = model.recommend()

recommendations finished on 1000/8709 queries. users per second: 71058.1

recommendations finished on 2000/8709 queries. users per second: 73730

recommendations finished on 3000/8709 queries. users per second: 69814.5

recommendations finished on 4000/8709 queries. users per second: 65284.8

recommendations finished on 5000/8709 queries. users per second: 71708.2

recommendations finished on 6000/8709 queries. users per second: 71108.6

recommendations finished on 7000/8709 queries. users per second: 77348.1

recommendations finished on 8000/8709 queries. users per second: 79399.7

In [71]:
pwd

u'/Users/jhonasttanregalado/Documents/DataScience/bootcamp7/githubcrush/bootcamp007_project/Project5-Capstone/shopaholics/collabFiltPython'

In [72]:
my_list_of_items = pd.read_csv('../BX-CSV-Dump/books1000.csv')

In [73]:
my_list_of_items = gl.SArray(my_list_of_items['ISBN'])

In [74]:
similar_items = model.get_similar_items(my_list_of_items[:10], k=20)


In [75]:
similar_items

ISBN,similar,score,rank
0971880107,0385504209,0.0223214030266,1
0971880107,0446605239,0.0181818008423,2
0971880107,0316666343,0.0165441036224,3
0971880107,0345342968,0.0158730149269,4
0971880107,0375702709,0.0140845179558,5
0971880107,0316601950,0.0134048461914,6
0971880107,0142001740,0.0133333206177,7
0971880107,0439139600,0.0132012963295,8
0971880107,0425192733,0.011194050312,9
0971880107,0446610542,0.0110294222832,10


In [139]:
def ISBNlookup(i):
    return items[items['ISBN']==i]

In [140]:
ISBNlookup('0971880107')

Columns:
	ISBN	str
	authors	str
	categories	str
	imageThumbnail	str
	pageCount	int
	publishedDate	str
	publisher	str
	textSnippet	str
	title	str

Rows: Unknown

Data:
+------------+--------------+------------+-------------------------------+-----------+
|    ISBN    |   authors    | categories |         imageThumbnail        | pageCount |
+------------+--------------+------------+-------------------------------+-----------+
| 0971880107 | Rich Shapero |  Fiction   | http://books.google.com/bo... |    315    |
+------------+--------------+------------+-------------------------------+-----------+
+---------------+-------------+-------------------------------+-------------+
| publishedDate |  publisher  |          textSnippet          |    title    |
+---------------+-------------+-------------------------------+-------------+
|      2004     | Too Far Pub | Newly graduated from colle... | Wild Animus |
+---------------+-------------+-------------------------------+-------------+
[? rows x 9 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.

In [82]:
model.save('my_model')

In [83]:
model = gl.load_model('my_model')

In [84]:
model.get

<bound method ItemSimilarityRecommender.get of Class                            : ItemSimilarityRecommender

Schema
------
User ID                          : UserID
Item ID                          : ISBN
Target                           : None
Additional observation features  : 0
User side features               : []
Item side features               : []

Statistics
----------
Number of observations           : 20557
Number of users                  : 8709
Number of items                  : 826

Training summary
----------------
Training time                    : 1.0461

Model Parameters
----------------
Model class                      : ItemSimilarityRecommender
threshold                        : 0.001
similarity_type                  : jaccard
training_method                  : auto

Other Settings
--------------
degree_approximation_threshold   : 4096
sparse_density_estimation_sample_size : 4096
max_data_passes                  : 4096
target_memory_usage              : 8589934592


## Visualization

In [85]:
type(items)

graphlab.data_structures.sframe.SFrame

In [86]:
validation_data.head()
type(validation_data)

graphlab.data_structures.sframe.SFrame

In [104]:
user_data
items.unique()

BookRating,ISBN,authors,categories,imageThumbnail,pageCount
0,0345348109,Michael Shaara,Fiction,http://books.google.com/books/content?id=rmyZj ...,355
8,089480829X,"Arlene Eisenberg, HeidiEisenberg Murkoff, Sa ...","Family & Relationships,Health & Fitness ...",http://books.google.com/books/content?id=rJhFd ...,454
9,0743224574,Sylvia Nasar,Biography & Autobiography,http://books.google.com/books/content?id=NDcwL ...,461
7,0446604402,Scott Turow,Fiction,http://books.google.com/books/content?id=pojCB ...,832
9,0425143325,Tom Clancy,Fiction,http://books.google.com/books/content?id=Ag64j ...,750
9,000649840X,Frank McCourt,English fiction,http://books.google.com/books/content?id=P_17n ...,426
8,0345348109,Michael Shaara,Fiction,http://books.google.com/books/content?id=rmyZj ...,355
7,0671024094,Jeffery Deaver,,http://books.google.com/books/content?id=zEkvO ...,560
10,0345413350,"Philip Pullman, PhilipPullman ...","Juvenile Fiction,Juvenile Fiction ...",http://books.google.com/books/content?id=cvbXm ...,351
9,1857029402,Annie Proulx,American literature,http://books.google.com/books/content?id=v0QgA ...,58


In [111]:
#grap user data from validation data set
#user_data = gl.SFrame(validation_data['UserID'])
user_data = gl.SFrame(dfRating['UserID'])
user_data.rename({'X1':'UserID'})
user_data = user_data.unique()
user_data.head()
user_data.unique


<bound method SFrame.unique of Columns:
	UserID	int

Rows: 8836

Data:
+--------+
| UserID |
+--------+
| 262565 |
| 241146 |
| 184486 |
| 162626 |
| 193084 |
| 180799 |
| 256889 |
| 250981 |
| 261460 |
| 235599 |
+--------+
[8836 rows x 1 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.>

In [90]:
user_data['UserID']

dtype: int
Rows: 493
[276754, 277517, 277533, 277915, 278162, 278582, 278582, 388, 638, 638, 638, 638, 638, 638, 643, 882, 882, 1008, 1652, 2179, 2179, 3917, 3917, 161173, 161299, 161662, 161870, 161882, 161937, 161995, 162030, 162052, 162052, 162052, 162519, 162562, 162590, 162590, 162626, 162671, 162671, 162671, 162738, 162738, 162738, 162738, 162827, 162827, 163169, 163326, 163326, 163485, 163570, 163570, 163570, 163570, 163600, 163804, 163804, 163804, 164270, 164549, 164557, 164956, 167689, 167746, 167759, 167811, 168275, 168383, 169341, 169400, 169496, 169735, 169735, 169885, 170256, 170634, 172030, 172030, 172168, 172742, 172742, 172742, 172742, 173486, 173679, 173679, 174914, 174914, 174914, 175739, 176054, 176459, 177090, 177091, 177091, 178056, 178079, 179327, ... ]

In [91]:
print validation_data['ISBN']
items['ISBN']

['0684867621', '0452282152', '0452268060', '0452269571', '059035342X', '0312983298', '0312990456', '006101351X', '0316693006', '0316776963', '0316789089', '0385504209', '0446523569', '0670892963', '0552137030', '0515135062', '0679781587', '0312195516', '0385504209', '0385504209', '0451524934', '0156007754', '0688156819', '0553573403', '0515135062', '0515136530', '0440234743', '034540288X', '0316776963', '0767900383', '044023722X', '006101351X', '0375725784', '0553271636', '0743437640', '0440213525', '0064407667', '0312291639', '0345337662', '0385730586', '0671027344', '0689817851', '0345339711', '0345445848', '0375703764', '0385504209', '0385497466', '0515136379', '0446605239', '0312201656', '0440226430', '0440235596', '0451163524', '0451172817', '067102423X', '0743457358', '0446365386', '0142001740', '0316666343', '0452282195', '0515132020', '0385504209', '0671510053', '0786868716', '044021145X', '0446608815', '0394820371', '055356451X', '0380002930', '0553582135', '0671026011', '0451

dtype: str
Rows: 55655
['000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '000649840X', '0020418604', '0020418604', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730X', '002542730

In [92]:
items.column_names

<bound method SFrame.column_names of Columns:
	UserID	int
	ISBN	str
	BookRating	int
	title	str
	authors	str
	publishedDate	str
	publisher	str
	categories	str
	pageCount	int
	imageThumbnail	str
	textSnippet	str

Rows: 55655

Data:
+--------+------------+------------+----------------+---------------+---------------+
| UserID |    ISBN    | BookRating |     title      |    authors    | publishedDate |
+--------+------------+------------+----------------+---------------+---------------+
|  901   | 000649840X |     9      | Angela's Ashes | Frank McCourt |      1997     |
| 164096 | 000649840X |     7      | Angela's Ashes | Frank McCourt |      1997     |
| 168646 | 000649840X |     9      | Angela's Ashes | Frank McCourt |      1997     |
| 168816 | 000649840X |     0      | Angela's Ashes | Frank McCourt |      1997     |
| 170227 | 000649840X |     0      | Angela's Ashes | Frank McCourt |      1997     |
| 173018 | 000649840X |     0      | Angela's Ashes | Frank McCourt |      1997   

In [112]:
view = model.views.overview(validation_set = validation_data,
                           user_data = user_data,
                           user_name_column = 'UserID',
                           item_data = items,
                           item_name_column = 'title',
                           item_url_column = 'imageThumbnail')
view.show()

Warning: The model was fit with 2 feature columns but only 1 were present during transform(). Proceeding with transform by ignoring the missing columns.

Warning: The model was fit with 2 feature columns but only 1 were present during transform(). Proceeding with transform by ignoring the missing columns.

In [ ]:
type(user_data)

## Build a predictive model

In [124]:
#build a model that predicts using ratings

training_data_2, validation_data_2 = gl.recommender.util.random_split_by_user(actions, 'UserID', 'ISBN')
model_predict_rating = gl.recommender.create(training_data_2, 'UserID', 'ISBN', target= 'BookRating')

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 20557 observations with 8709 users and 826 items.

Data prepared in: 0.069258s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.96212

Final training RMSE: 0.626594

In [125]:
results_predict_rating = model_predict_rating.recommend()

recommendations finished on 1000/8709 queries. users per second: 87581

recommendations finished on 2000/8709 queries. users per second: 100644

recommendations finished on 3000/8709 queries. users per second: 105141

recommendations finished on 4000/8709 queries. users per second: 105907

recommendations finished on 5000/8709 queries. users per second: 107717

recommendations finished on 6000/8709 queries. users per second: 104513

recommendations finished on 7000/8709 queries. users per second: 101184

recommendations finished on 8000/8709 queries. users per second: 98259.6

In [141]:
model_predict_rating.save('my_model_predict_rating')

In [142]:
similar_items_predict_rating = model_predict_rating.get_similar_items(my_list_of_items[:10], k=20)



In [143]:
print similar_items
similar_items_predict_rating

+------------+------------+-----------------+------+
|    ISBN    |  similar   |      score      | rank |
+------------+------------+-----------------+------+
| 0971880107 | 0385504209 | 0.0223214030266 |  1   |
| 0971880107 | 0446605239 | 0.0181818008423 |  2   |
| 0971880107 | 0316666343 | 0.0165441036224 |  3   |
| 0971880107 | 0345342968 | 0.0158730149269 |  4   |
| 0971880107 | 0375702709 | 0.0140845179558 |  5   |
| 0971880107 | 0316601950 | 0.0134048461914 |  6   |
| 0971880107 | 0142001740 | 0.0133333206177 |  7   |
| 0971880107 | 0439139600 | 0.0132012963295 |  8   |
| 0971880107 | 0425192733 |  0.011194050312 |  9   |
| 0971880107 | 0446610542 | 0.0110294222832 |  10  |
+------------+------------+-----------------+------+
[180 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


ISBN,similar,score,rank
0971880107,0452267455,0.730154633522,1
0971880107,042517736X,0.673560261726,2
0971880107,0688156819,0.670377969742,3
0971880107,1573221937,0.663085877895,4
0971880107,0316089699,0.657558619976,5
0971880107,0451526503,0.647027730942,6
0971880107,0425146413,0.635923624039,7
0971880107,0671004565,0.61257737875,8
0971880107,0553569031,0.599737167358,9
0971880107,0452272726,0.5822250247,10


In [136]:
ISBNlookup('0971880107')

Columns:
	ISBN	str
	authors	str
	categories	str
	imageThumbnail	str
	pageCount	int
	publishedDate	str
	publisher	str
	textSnippet	str
	title	str

Rows: Unknown

Data:
+------------+--------------+------------+-------------------------------+-----------+
|    ISBN    |   authors    | categories |         imageThumbnail        | pageCount |
+------------+--------------+------------+-------------------------------+-----------+
| 0971880107 | Rich Shapero |  Fiction   | http://books.google.com/bo... |    315    |
+------------+--------------+------------+-------------------------------+-----------+
+---------------+-------------+-------------------------------+-------------+
| publishedDate |  publisher  |          textSnippet          |    title    |
+---------------+-------------+-------------------------------+-------------+
|      2004     | Too Far Pub | Newly graduated from colle... | Wild Animus |
+---------------+-------------+-------------------------------+-------------+
[? rows x 9 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.

In [145]:
type(validation_data_2)

graphlab.data_structures.sframe.SFrame

In [146]:
view_predicted_model = model_predict_rating.views.overview(validation_set = validation_data_2,
                           user_data = user_data,
                           user_name_column = 'UserID',
                           item_data = items,
                           item_name_column = 'title',
                           item_url_column = 'imageThumbnail')
view_predicted_model.show()

Warning: The model was fit with 2 feature columns but only 1 were present during transform(). Proceeding with transform by ignoring the missing columns.

Warning: The model was fit with 2 feature columns but only 1 were present during transform(). Proceeding with transform by ignoring the missing columns.